# Advanced Dataframes

## Agenda

1. Part I: Creating Dataframes
    - from Lists, Arrays, & Dictionaries
    - from PyDataset
    - from a SQL query
2. Exercises, Part I
3. Part II
    - Indexing and Subsetting
    - Aggregating
    - Merging & Joining
4. Exercises, Part II
5. Part III: Reshaping & Transposing
    - Reshaping
    - Transposing
10. Exercises III

In [1]:
# imports
import pandas as pd
import numpy as np

## Part I: Creating Dataframes

### From Lists

In [2]:
pd.DataFrame([[1,2,3],[4,5,6]])

,0,1,2
0,1,2,3
1,4,5,6


### From Arrays

In [3]:
a = np.array([[1,2,3],[4,5,6]])
pd.DataFrame(a, columns = ['a','b','c'])

,a,b,c
0,1,2,3
1,4,5,6


### From Dictionaries

In [4]:
pd.DataFrame({'a':[1,4],
             'b':[2,5],
             'c':[3,6]
             })

,a,b,c
0,1,2,3
1,4,5,6


### From PyDataset

In [5]:
from pydataset import data

### From SQL

We can use the read_sql method to create a dataframe based on the results of a SQL query. To do this, we need to tell pandas how to connect to the database we are querying. The way we communicate this to pandas is with a specially formatted connection string.

In addition, whenever we want to connect to a database from our python code (other programming languages are similar), we will need a driver, a bit of software that handles the details of the database connection.

1. In order to connect to mysql, we'll install the pymysql driver packages by typing the following in your terminal: `python -m pip install pymysql'
1. **Add env.py to the repo's .gitignore file**
2. create a text file called env.py, and in your text file, write the following (filling in the corresponding info):

```python
host = '<ip_address>'
username = '<your_username>'
password = '<your_password>'
```

4. import the host, username, and password variables from the env.py file. Note that upon import, you use `env`, not `env.py`. **once you assign the string to the variable `url`, DO NOT print the value of url to your notebook. If you do this and push the notebook to github, your username and password will be visible to others.**

In [14]:
from pydataset import data
import pandas as pd
import numpy as np
from env import host, username, password 

# create the url variable (string) with my username, password and host inserted
url = f'mysql+pymysql://{username}:{password}@{host}/employees'   
    


In [15]:
# write the SQL query to gather data, assign to variable 'query'
query = 'SELECT * FROM employees LIMIT 10'

In [16]:
df = pd.read_sql(query, url)
# read the data passing the function the query and the url string

In [17]:
# write a query in multiple lines:
df.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [ ]:
query = '''

SELECT
    emp_no,
    first_name,
    last_name,
FROM employees
WHERE gender = 'F'
LIMIT 100;
'''

employees_df = pd.read_sql(query, url)
employees_df.head()

## Exercises, Part I

Create a notebook or python script named `advanced_dataframes` to do your work in for these exercises.

1. Run python -m pip install pymysql from your terminal to install the mysql client (any folder is fine)

2. cd into your exercises folder for this module and run echo env.py >> .gitignore

3. Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

4. Use your function to obtain a connection to the employees database.

5. Once you have successfully run a query:
    - Intentionally make a typo in the database url. What kind of error message do you see?
    - Intentionally make an error in your SQL query. What does the error message look like?


6. Read the employees and titles tables into two separate DataFrames.

7. How many rows and columns do you have in each DataFrame? Is that what you expected?

8. Display the summary statistics for each DataFrame.

9. How many unique titles are in the titles DataFrame?

10. What is the oldest date in the to_date column?

11. What is the most recent date in the to_date column?

In [ ]:
import os
import env 
def get_db_url(db, env_file = os.path.exists('env.py')):
    '''
    this will return a formatted string containing username, password, host, and database
    for connecting to the mysql server and the database indicated.
    env_file checks to see if the env.py exists in the cwd
    
    '''
    if env_file:
        username, password, host = (env.username, env.password, env.host)
        return f'mysql+pymysql://{username}:{password}@{host}/{db}'
    else:
        return 'you need to a username'
    
url = get_db_url('employees')

## Part II

### Indexing and Subsetting

- `[]`: subset rows using a boolean mask, or subset columns using a list of column names. 
- `.loc`: allows for subsetting rows and columns simultaneously, using the labels/names of rows/columns. 
- `.iloc`: allows for subsetting rows and columns simultaneously, using the index location or position of rows/columns. 

In [ ]:
# Create list of values for names column.
import pandas as pd
import numpy as np

np.random.seed(123)

names = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas', 'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']


In [ ]:
# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=101, size=len(names))
english_grades = np.random.randint(low=60, high=101, size=len(names))
reading_grades = np.random.randint(low=60, high=101, size=len(names))

classroom = np.random.choice(['A','B'], len(names))

In [ ]:
# Construct the DataFrame using the above lists and arrays.


df = pd.DataFrame({'names': names,
                    'math': math_grades,
                      'english': english_grades,
                      'reading': reading_grades,
                      'classroom': classroom
                      })

Bracket: `[]`

In [ ]:
# select 2 columns from the class
df[['names','classroom']]

In [ ]:
# I can pass a boolean Series to the indexing operator as a selector.
bools = df.names.str.startswith('A')

In [ ]:
# then subset using brackets with the boolean series

`.loc`

- Select specific rows AND columns by index **label**. 
- The index label can be a number, but it can also be a string label. 
- It is **inclusive.**

`df.loc[row_indexer, column_indexer]`

In [ ]:
# Select all the rows and a subset of columns; 
# notice the inclusive behavior of the indexing.
df.loc[:,'math':'reading'].head(5)

In [ ]:
# I can use a boolean Series as a selector with .loc, too, 
# but I can choose rows and columns.
df.loc[bools, 'math':'reading']

`.iloc`

- Select specific rows and colums by index **position**. 
- It does **not** accept a boolean Series as a selector like .loc does. 
- It takes in integers representing index position
- It is **exclusive**.

`df.iloc[row_indexer, column_indexer]`

In [ ]:
# select rows by integer position
# notice the exclusive behavior of the indexing

In [ ]:
df.iloc[:4, 2:5]

### Aggregating

`.agg`

- Specify a way to aggregate a series of numerical values. 
- We pass an aggregate function or list of functions to the method that we want applied to a Series.

In [18]:
df.reading.agg('min')

AttributeError: 'DataFrame' object has no attribute 'reading'

What happens if you run it on the entire dataframe, not just a series of a single column? 

In [ ]:
# aggregate each column in the dataframe by taking the min value
df[['math','english','reading']].agg(['mean', 'min','max']).round(2)

While on the surface this seems pretty simple, `.agg` is capable of providing more detailed aggregations:

`groupby`

- Creates a grouped object. 
- Then apply an aggregation on that object. 

For example, if we wanted to know the highest math grade from each classroom:

In [ ]:
df.groupby('classroom').math.agg(['mean'])

We can use `.agg` here to, to see multiple aggregations:

In [ ]:
df.groupby('classroom')[['math','english']].agg(['mean',
                                                    'median',
                                                    'max',
                                                    'min'
                                                    ])

#nested columns

We can group by multiple columns as well. 

Let's compute the average reading grade for students who are passing english and those who are not passing english. 


In [ ]:
# Create a boolean column named passing_english 
# based on the math column using np.where
df['passing_math'] = np.where(df.math < 70, 'failing', 'passing')
df

In [ ]:
# Group by the combination of our new feature, passing_english,
# and the classroom.
# aggregate by calculating average reading grade
grade_groups = df.groupby(['classroom','passing_math']).reading.agg(['mean','count'])

In [ ]:
# clean up the column names to make the data more transparent
grade_groups.index
grade_groups.columns = ['avg_reading_grade', 'number_of_students']
grade_groups

`.transform`

When you want to add an aggregate value, like the average math grades for each classroom, back to the original dataframe. 

In [ ]:
df.groupby('classroom').math.agg('mean')

In [ ]:
df.assign(avg_math_by_classroom = df.groupby('classroom').math.transform('mean'))
#transform works to spread it across the dataframe

In [ ]:
df

`.describe` 

Can be used with `groupby` also. 

In [ ]:
df.groupby('classroom').describe().T

### Merging & Joining

Ways to combine dataframes together

`pd.concat`

- This function takes in a list or dictionary of Series or DataFrame objects and joins them along a particular axis
- Row-wise: axis=0, means your new data frame will be longer with more rows. 
- Column-wise: axis=1, means your new dataframe will be wider with more columns. 
- When your list contains at least one DataFrame, a DataFrame is returned.
- When concatenating only Series objects row-wise, axis=0, a Series is returned.
- When concatenating Series or DataFrames column-wise, axis=1, a DataFrame is returned.

default parameters: 

`pd.concat(objs_in_list, axis=0, join='outer')`

In [ ]:
# concatenate 2 dataframes by passing a list of the 2 dfs, and 
# indicating whether to concatenate along the rows or the columns

In [ ]:
df.shape

In [ ]:
df1 = pd.DataFrame({'a': [1,2,3]})
df2 = pd.DataFrame({'a': [98,6,5]})
pd.concat([df1,df2], axis=0)

`.merge`

- Similar to a SQL join.
- first dataframe is the 'left' table. 
- `how`: Type of merge to be performed.
    - `how=left`: use only keys from left frame, similar to a SQL left outer join; preserve key order.
    - `how=right`: use only keys from right frame, similar to a SQL right outer join; preserve key order.
    - `how=outer`: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.
    - `how=inner`: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.


In [ ]:
# Create the users DataFrame.
users = pd.DataFrame({
    'id':[1,2,3,4,5,6],
    'name':['bob','joe','sally','adam','jorge', 'mike'],
    'role_id':[1,2,3,3,np.nan,np.nan]
    
    
})
users

In [ ]:
# Create the roles DataFrame
roles = pd.DataFrame({
    'id': [1,2,3,4],
    'name':['admin','author','reviewer','commenter']
})
roles

- `left_on` and `right_on`: indicate the columns that are the keys used to merge the dataframes together.

- `indicator`=`True` will create a column indicating whether the merge key appears in the `left_only`, `right_only` or `both` DataFrames.

In [ ]:
# Perform an outer join specifying the left and right DataFrame keys.
users.merge(roles, left_on='role_id', right_on = 'id', how='outer',indicator = True)

- suffix of `_x` to any columns in the left dataframe that are duplicated, 
- suffix of `_y` to any columns in the right dataframe that are duplicated. 
- can clean up column names using **method chaining**.

In [ ]:
(users.merge(roles, left_on='role_id', right_on = 'id', how='outer',indicator = True)
             .drop(columns = 'role_id')
            .rename(columns={'id_x':'id',
                            'name_x':'employee',
                             'id_y':'role_id',
                             'name_y':'role'}))

## Part 3: Reshaping and Transposing

1. Reshaping: pd.crosstab, .pivot_table
2. Transposing

### Reshaping

`pd.crosstab`

Count the number of students passing math in each classroom (student grades df):

In [ ]:
df

In [ ]:
pd.crosstab(df.passing_math,df.classroom)

Add subtotals using the `margins` argument:

In [ ]:
pd.crosstab(df.passing_math, df.classroom,
            margins=True)

View percentages instead of actual values using the `normalize` argument. 

In [ ]:
pd.crosstab(df.passing_math, df.classroom,
            margins=True, normalize ='columns').round(2)

`.pivot_table`

Like an excel pivot table

Supply 4 pieces of info to arguments: 
- which values will make up the rows, the `index`
- which values will make up the columns
- the values we are aggregating
- an aggregation method (`aggfunc`) (default is `mean`). 

Example: Calculate the average math grade for the combination of `classroom` and `passing_math` status. 

In [ ]:
df.pivot_table(index='classroom', 
               columns= 'passing_math', 
               values = 'math', 
               aggfunc= ['mean','sum'])

Create a new dataframe: orders at a restaurant. 

In [ ]:
n = 40

orders = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})

orders.sample(10)


`.map`

Use a dictionary to calculate the total price for an order. Then save the calculations to a new column named `bill`. 

In [ ]:


# Create a dictionary of prices for drinks and meals.

prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}


In [ ]:
# match the values in the drink and meal columns with the values 
# in the prices and perform the specificied calculations. 
# save to a new column, bill

orders['bill'] = orders.drink.map(prices) + orders.meal.map(prices)
orders.head()

In [ ]:
# use crosstab to look at how many orders have each combination of
# meal and drink
pd.crosstab(orders.drink, orders. meal)

In [ ]:
# find the average bill for each combination using pivot_table
orders.pivot_table(index='drink', 
                   columns='meal', 
                   values='bill',
                    aggfunc = 'mean'
                  )

In [ ]:
# find the average bill for each combination using groupby

Transposing

Swapping rows and columns

In [ ]:
orders.groupby(['drink','meal'])['bill'].agg('mean')